# 🛒 titre

|   |   |
|---|---|
| Project  |    [nom du proj](#https://openclassrooms.com/fr/paths/148/projects/630/assignment)         |
| Date   |   February 2023   |
| Autor  | Ana Bernal                                                    |
| Data source | |
| Mentor | Samir Tanfous | 
| Notebook number  | _ of _                                                     |

**Description:** blabla

# Preliminaries
## Imports

In [1]:
import pandas as pd
import numpy as np

## Graphics and options

## Data

In [2]:
path = './data/'
filename = 'QueryResults.csv'

df = pd.read_csv(path + filename)

In [3]:
df.shape

(50000, 7)

In [4]:
df.head()

,Title,Body,Tags,Id,Score,ViewCount,AnswerCount
0,Multithreading in a stateless session bean?,<p>The EJB 3.0 specification does not allow a ...,<java><multithreading><jakarta-ee><ejb-3.0><ejb>,3816286,9,13503,4
1,base64 JSON encoded strings in nodejs,<p>How do I create a base64 JSON encoded strin...,<javascript><json><node.js><base64><buffer>,22515180,14,23710,3
2,How do you get a directory listing in C?,<p>How do you scan a directory for folders and...,<c><file><directory><cross-platform><common-ta...,12489,69,142856,9
3,"Link error: ""Cannot find -ltinfo"" on Ubuntu 12...",<p>I get the following link error when I'm com...,<c++><ubuntu><arm><static-libraries><ubuntu-12...,21137879,21,17090,2
4,Android In App Updates - Not able to detect th...,"<p>Some log statements are thrown when we use,...",<android><google-play><google-play-services><a...,56581300,9,5122,3


# Cleaning Tags column, counting tags

In [5]:
def simple_tokenize_tags(tag):
    '''
    Returns a list with more readable original 
    tags from StackOverflow. 

    Parameters:
        - tag: str


    Example:
    tag = '<java><multithreading><jakarta-ee><ejb-3.0><ejb>'

    simple_tokenize_tags(tag)
    ['java', 'multithreading', 'jakarta-ee', 'ejb-3.0', 'ejb']

    '''
    return tag.replace('><', ' ')[1:-1].split()

In [6]:
# Adding column counting tags to df
df['nb_tags'] = df['Tags'].apply(lambda tags : len(simple_tokenize_tags(tags)))

In [7]:
df.nb_tags.describe()

count    50000.000000
mean         5.000040
std          0.006324
min          5.000000
25%          5.000000
50%          5.000000
75%          5.000000
max          6.000000
Name: nb_tags, dtype: float64

In [9]:
df.loc[df.nb_tags == 6].shape

(2, 8)

We queried questions with more than 5 tags. Since 5 is the maximum number of tags allowed by Stack Overflow per question, we only have (almost) 5 tag questions.

# Defining tags set

In [16]:
df.head(3)

,Title,Body,Tags,Id,Score,ViewCount,AnswerCount,nb_tags
0,Multithreading in a stateless session bean?,<p>The EJB 3.0 specification does not allow a ...,<java><multithreading><jakarta-ee><ejb-3.0><ejb>,3816286,9,13503,4,5
1,base64 JSON encoded strings in nodejs,<p>How do I create a base64 JSON encoded strin...,<javascript><json><node.js><base64><buffer>,22515180,14,23710,3,5
2,How do you get a directory listing in C?,<p>How do you scan a directory for folders and...,<c><file><directory><cross-platform><common-ta...,12489,69,142856,9,5
3,"Link error: ""Cannot find -ltinfo"" on Ubuntu 12...",<p>I get the following link error when I'm com...,<c++><ubuntu><arm><static-libraries><ubuntu-12...,21137879,21,17090,2,5
4,Android In App Updates - Not able to detect th...,"<p>Some log statements are thrown when we use,...",<android><google-play><google-play-services><a...,56581300,9,5122,3,5


In [17]:
df['clean_Tags'] = df['Tags'].apply(lambda txt:simple_tokenize_tags(txt))

In [18]:
df.head(3)

,Title,Body,Tags,Id,Score,ViewCount,AnswerCount,nb_tags,clean_Tags
0,Multithreading in a stateless session bean?,<p>The EJB 3.0 specification does not allow a ...,<java><multithreading><jakarta-ee><ejb-3.0><ejb>,3816286,9,13503,4,5,"[java, multithreading, jakarta-ee, ejb-3.0, ejb]"
1,base64 JSON encoded strings in nodejs,<p>How do I create a base64 JSON encoded strin...,<javascript><json><node.js><base64><buffer>,22515180,14,23710,3,5,"[javascript, json, node.js, base64, buffer]"
2,How do you get a directory listing in C?,<p>How do you scan a directory for folders and...,<c><file><directory><cross-platform><common-ta...,12489,69,142856,9,5,"[c, file, directory, cross-platform, common-ta..."
3,"Link error: ""Cannot find -ltinfo"" on Ubuntu 12...",<p>I get the following link error when I'm com...,<c++><ubuntu><arm><static-libraries><ubuntu-12...,21137879,21,17090,2,5,"[c++, ubuntu, arm, static-libraries, ubuntu-12..."
4,Android In App Updates - Not able to detect th...,"<p>Some log statements are thrown when we use,...",<android><google-play><google-play-services><a...,56581300,9,5122,3,5,"[android, google-play, google-play-services, a..."


In [19]:
# The set of all tags appearing in the sample of
# size 50 000
all_tags = set()

for tags in df['clean_Tags']:
    all_tags = all_tags.union(set(tags))

In [20]:
print(f'There are {len(all_tags)} different tags.')

18409

In [38]:
# Different tags as a list
all_tags = list(all_tags)

# Counting questions per tag
all_tags_count = dict(zip(all_tags,np.zeros(len(all_tags))))

for tags in df['clean_Tags']:
    for tag in tags:
        all_tags_count[tag] += 1

# Array counting # of questions per tag
tags_count = np.array(list(all_tags_count.values()))

print(f'The mean of # of questions per tag is: {np.mean(tags_count)}')
print(f'The minimum # of questions per tag is: {np.min(tags_count)} (by constr.)')
print(f'The max # of questions per tag is: {np.max(tags_count)}.')
print(f'25% of tags appear in {np.quantile(tags_count, 0.25)} question.')
print(f'50% of tags appear in {np.quantile(tags_count, 0.5)} questions or less.')
print(f'75% of tags appear in {np.quantile(tags_count, 0.75)} questions or less.')


The mean of # of questions per tag is: 13.58042261937096
The minimum # of questions per tag is: 1.0 (by constr.)
The max # of questions per tag is: 6226.0.
25% of tags appear in 1.0 question.
50% of tags appear in 2.0 questions or less.
75% of tags appear in 6.0 questions or less.


In [122]:
numb_quest = 800
popular_tags = { key:value for (key,value) in all_tags_count.items() if value >numb_quest}

# Sorting popular tags decreasingly
popular_tags = {k: v for k, v in sorted(popular_tags.items(), key=lambda item: item[1], reverse=True)}

print(f'The total # of popular tags is {len(popular_tags)}')
print(f'(appearing in more than {numb_quest} questions)')
print('-'*35)
print('Tag \t\t # of quest')
print('-'*35)
for (key,value) in popular_tags.items():
    print(f'{key:>10} \t {value}')


The total # of popular tags is 30
(appearing in more than 800 questions)
-----------------------------------
Tag 		 # of quest
-----------------------------------
        c# 	 6226.0
      java 	 5880.0
javascript 	 4920.0
    python 	 4747.0
       c++ 	 3904.0
       ios 	 3449.0
   android 	 3223.0
      .net 	 3164.0
      html 	 2197.0
       php 	 2050.0
objective-c 	 1816.0
    jquery 	 1810.0
         c 	 1655.0
    iphone 	 1592.0
       sql 	 1349.0
   asp.net 	 1339.0
       css 	 1331.0
     linux 	 1286.0
   node.js 	 1253.0
performance 	 1076.0
    spring 	 1023.0
   windows 	 1012.0
     swift 	 986.0
     xcode 	 972.0
ruby-on-rails 	 956.0
     mysql 	 903.0
      json 	 866.0
sql-server 	 859.0
multithreading 	 834.0
asp.net-mvc 	 829.0


In [126]:
popular_tags.keys()

dict_keys(['c#', 'java', 'javascript', 'python', 'c++', 'ios', 'android', '.net', 'html', 'php', 'objective-c', 'jquery', 'c', 'iphone', 'sql', 'asp.net', 'css', 'linux', 'node.js', 'performance', 'spring', 'windows', 'swift', 'xcode', 'ruby-on-rails', 'mysql', 'json', 'sql-server', 'multithreading', 'asp.net-mvc'])

In [138]:
'java' in popular_tags.keys()

True

In [137]:
# conditions = []
# for tag in list(popular_tags.keys()):
#     conditions.append(df['Tags'].str.contains(tag))

# df.loc['javascript' in df['clean_Tags']]

In [139]:
df['contain_pop_tag'] = df.clean_Tags.explode().isin(popular_tags.keys()).groupby(level=0, sort=False).any()


In [141]:
df.contain_pop_tag.sum()

41193

In [144]:
df.loc[df.contain_pop_tag].shape

(41193, 11)

In [140]:
df.head()

,Title,Body,Tags,Id,Score,ViewCount,AnswerCount,nb_tags,clean_Tags,no_emoj_Title,contain_pop_tag
0,Multithreading in a stateless session bean?,<p>The EJB 3.0 specification does not allow a ...,<java><multithreading><jakarta-ee><ejb-3.0><ejb>,3816286,9,13503,4,5,"[java, multithreading, jakarta-ee, ejb-3.0, ejb]",Multithreading in a stateless session bean?,True
1,base64 JSON encoded strings in nodejs,<p>How do I create a base64 JSON encoded strin...,<javascript><json><node.js><base64><buffer>,22515180,14,23710,3,5,"[javascript, json, node.js, base64, buffer]",base64 JSON encoded strings in nodejs,True
2,How do you get a directory listing in C?,<p>How do you scan a directory for folders and...,<c><file><directory><cross-platform><common-ta...,12489,69,142856,9,5,"[c, file, directory, cross-platform, common-ta...",How do you get a directory listing in C?,True
3,"Link error: ""Cannot find -ltinfo"" on Ubuntu 12...",<p>I get the following link error when I'm com...,<c++><ubuntu><arm><static-libraries><ubuntu-12...,21137879,21,17090,2,5,"[c++, ubuntu, arm, static-libraries, ubuntu-12...","Link error: ""Cannot find -ltinfo"" on Ubuntu 12...",True
4,Android In App Updates - Not able to detect th...,"<p>Some log statements are thrown when we use,...",<android><google-play><google-play-services><a...,56581300,9,5122,3,5,"[android, google-play, google-play-services, a...",Android In App Updates - Not able to detect th...,True


# Processing title and body text

In [ ]:
import spacy

# Instantiating language model, english
nlp = spacy.load("en_core_web_sm")

2023-02-06 11:48:57.722275: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-06 11:48:57.917117: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-06 11:48:57.917149: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-06 11:48:58.915138: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [59]:
from bs4 import BeautifulSoup

In [83]:
def clean(text,tokenize,strict, **kwargs):
    """
    Returns a dictionnary with keys 'text' or 'tokens', where
    'tokens' corresponds tothe list of lemmatized tokens from
    the string text. Ommiting stopwords and punctuation, and the text is
    the joint text.

    Parameters:
        - text: str
        - tokenize: bool
            If True returns list of tokens, if False returns string.
        - strict: bool
            If true only keeps nouns
    """

    # Removing HTML tags
    soup = BeautifulSoup(text, features='html.parser')
    clean_txt = soup.get_text()

    # Removing new line character: \n
    clean_txt = clean_txt.replace('\n', ' ')

    clean_txt = clean_txt.replace('\ud83c', ' ')
    
    # Removing digits
    clean_txt = ''.join(char for char in clean_txt if not char.isdigit())

    # Tokenize with spacy
    doc = nlp(clean_txt)

    # Tokenize properties
    if strict == True:
        tokens = {token.lemma_.lower() :token.pos_ for token in doc
                    if token.pos_ in ['NOUN', 'PROPN'] and
                        (not (token.is_stop or 
                              token.is_punct or
                              token.is_space
                              )
                        )
                }    
    else:
        tokens = {token.lemma_.lower() :token.pos_ for token in doc
                    if not (token.is_stop or 
                            token.is_punct or
                            token.is_space
                            )
                }

    clean_txt = ' '.join(list(tokens.keys()))
    
    # Ask if return text or tokens
    if tokenize == True:
        result = tokens
    else:
        result = clean_txt

    # If list of entities in output
    if 'ent' in kwargs:
        result = {'output':result, 'ents': doc.ents}

    return result

Let us test this function with two examples:

**Example 1:**

In [88]:
# Text for example
text_ex = df.loc[37,'Body']
text_ex

"<p>I've noticed that a few Wordpress blogs have query statistics present in their footer that simply state the number of queries and the total time required to process them for the particular page, reading something like:</p>\n\n<blockquote>\n  <p>23 queries. 0.448 seconds</p>\n</blockquote>\n\n<p>I was wondering how this is accomplished.  Is it through the use of a particular Wordpress plug-in or perhaps from using some particular php function in the page's code?</p>\n"

In [77]:
clean(text_ex, tokenize=False, strict=False)

'notice wordpress blog query statistic present footer simply state number total time require process particular page read like second wonder accomplish use plug php function code'

In [78]:
print(clean(text_ex, tokenize=True, strict=False))

{'notice': 'VERB', 'wordpress': 'ADJ', 'blog': 'NOUN', 'query': 'NOUN', 'statistic': 'NOUN', 'present': 'ADJ', 'footer': 'NOUN', 'simply': 'ADV', 'state': 'VERB', 'number': 'NOUN', 'total': 'ADJ', 'time': 'NOUN', 'require': 'VERB', 'process': 'VERB', 'particular': 'ADJ', 'page': 'NOUN', 'read': 'VERB', 'like': 'ADP', 'second': 'NOUN', 'wonder': 'VERB', 'accomplish': 'VERB', 'use': 'NOUN', 'plug': 'NOUN', 'php': 'NOUN', 'function': 'NOUN', 'code': 'NOUN'}


Tokens:

In [79]:
clean(text_ex, tokenize=True, strict=False).keys()

dict_keys(['notice', 'wordpress', 'blog', 'query', 'statistic', 'present', 'footer', 'simply', 'state', 'number', 'total', 'time', 'require', 'process', 'particular', 'page', 'read', 'like', 'second', 'wonder', 'accomplish', 'use', 'plug', 'php', 'function', 'code'])

Using the `strict=True` condition

In [84]:
clean(text_ex, tokenize=False, strict=True)

'blog query statistic footer number time page second use plug php function code'

In [86]:
print(clean(text_ex, tokenize=True, strict=True))

{'blog': 'NOUN', 'query': 'NOUN', 'statistic': 'NOUN', 'footer': 'NOUN', 'number': 'NOUN', 'time': 'NOUN', 'page': 'NOUN', 'second': 'NOUN', 'use': 'NOUN', 'plug': 'NOUN', 'php': 'NOUN', 'function': 'NOUN', 'code': 'NOUN'}


In [87]:
clean(text_ex, tokenize=True, strict=True).keys()

dict_keys(['blog', 'query', 'statistic', 'footer', 'number', 'time', 'page', 'second', 'use', 'plug', 'php', 'function', 'code'])

**Example 2:**

In [89]:
# Sample text
sample_text = df.loc[10787,'Body']
sample_text

'<p>I\'m learning Scala with the aim of writing agent based simulations using actor concurrency.  I currently know very little about OpenCL, and before I dive in can anyone tell me if it is likely to be appropriate/compatible with agent based simulations?  </p>\n\n<p>If so, then <a href="http://ochafik.free.fr/blog/?p=207" rel="nofollow noreferrer">ScalaCL</a> looks very attractive.</p>\n'

In [93]:
clean(sample_text, tokenize=False, strict=False)

'learn scala aim write agent base simulation actor concurrency currently know little opencl dive tell likely appropriate compatible scalacl look attractive'

In [94]:
print(clean(sample_text, tokenize=True, strict=False))

{'learn': 'VERB', 'scala': 'PROPN', 'aim': 'NOUN', 'write': 'VERB', 'agent': 'NOUN', 'base': 'VERB', 'simulation': 'NOUN', 'actor': 'NOUN', 'concurrency': 'NOUN', 'currently': 'ADV', 'know': 'VERB', 'little': 'ADJ', 'opencl': 'PROPN', 'dive': 'VERB', 'tell': 'VERB', 'likely': 'ADJ', 'appropriate': 'ADJ', 'compatible': 'ADJ', 'scalacl': 'NOUN', 'look': 'VERB', 'attractive': 'ADJ'}


Tokens:

In [95]:
clean(sample_text, tokenize=True, strict=False).keys()

dict_keys(['learn', 'scala', 'aim', 'write', 'agent', 'base', 'simulation', 'actor', 'concurrency', 'currently', 'know', 'little', 'opencl', 'dive', 'tell', 'likely', 'appropriate', 'compatible', 'scalacl', 'look', 'attractive'])

Using the `strict=True` condition

In [96]:
clean(sample_text, tokenize=False, strict=True)

'scala aim agent simulation actor concurrency opencl scalacl'

In [97]:
print(clean(sample_text, tokenize=True, strict=True))

{'scala': 'PROPN', 'aim': 'NOUN', 'agent': 'NOUN', 'simulation': 'NOUN', 'actor': 'NOUN', 'concurrency': 'NOUN', 'opencl': 'PROPN', 'scalacl': 'NOUN'}


In [98]:
clean(sample_text, tokenize=True, strict=True).keys()

dict_keys(['scala', 'aim', 'agent', 'simulation', 'actor', 'concurrency', 'opencl', 'scalacl'])

Entities: (NER)

In [92]:
clean(sample_text, tokenize=True, ent=True, strict=False)['ents']

(Scala,)

Adding clean column for title, body and tags

In [100]:
df.head(3)

,Title,Body,Tags,Id,Score,ViewCount,AnswerCount,nb_tags,clean_Tags
0,Multithreading in a stateless session bean?,<p>The EJB 3.0 specification does not allow a ...,<java><multithreading><jakarta-ee><ejb-3.0><ejb>,3816286,9,13503,4,5,"[java, multithreading, jakarta-ee, ejb-3.0, ejb]"
1,base64 JSON encoded strings in nodejs,<p>How do I create a base64 JSON encoded strin...,<javascript><json><node.js><base64><buffer>,22515180,14,23710,3,5,"[javascript, json, node.js, base64, buffer]"
2,How do you get a directory listing in C?,<p>How do you scan a directory for folders and...,<c><file><directory><cross-platform><common-ta...,12489,69,142856,9,5,"[c, file, directory, cross-platform, common-ta..."


In [101]:
df['clean_Title'] = df['Title'].apply(lambda txt:clean(txt, tokenize=False, strict=True))

UnicodeEncodeError: 'utf-8' codec can't encode character '\ud83d' in position 0: surrogates not allowed

Other approach, filtering dataframe.

In [145]:
df_popular_tags = df.loc[df.contain_pop_tag].copy()

In [146]:
df_popular_tags.head(3)

,Title,Body,Tags,Id,Score,ViewCount,AnswerCount,nb_tags,clean_Tags,no_emoj_Title,contain_pop_tag
0,Multithreading in a stateless session bean?,<p>The EJB 3.0 specification does not allow a ...,<java><multithreading><jakarta-ee><ejb-3.0><ejb>,3816286,9,13503,4,5,"[java, multithreading, jakarta-ee, ejb-3.0, ejb]",Multithreading in a stateless session bean?,True
1,base64 JSON encoded strings in nodejs,<p>How do I create a base64 JSON encoded strin...,<javascript><json><node.js><base64><buffer>,22515180,14,23710,3,5,"[javascript, json, node.js, base64, buffer]",base64 JSON encoded strings in nodejs,True
2,How do you get a directory listing in C?,<p>How do you scan a directory for folders and...,<c><file><directory><cross-platform><common-ta...,12489,69,142856,9,5,"[c, file, directory, cross-platform, common-ta...",How do you get a directory listing in C?,True


In [147]:
df_popular_tags['clean_Title'] = df_popular_tags['Title'].apply(lambda txt:clean(txt, tokenize=False, strict=True))

UnicodeEncodeError: 'utf-8' codec can't encode character '\ud83d' in position 0: surrogates not allowed

Work with sample of 10 000 questions since I have been having this error that haven't been able to solve:

```
UnicodeEncodeError: 'utf-8' codec can't encode character '\ud83d' in position 0: surrogates not allowed
```

In [150]:
df_popular_tags_sample = df_popular_tags.sample(10000)

# Export this sample to work with later
path = './data/'
filename = 'sample_questions.csv'

df_popular_tags_sample.to_csv(path + filename, index=False)

In [ ]:
# Import sample
# df_popular_tags_sample = pd.read_csv(path + filename)

In [151]:
df_popular_tags_sample['clean_Title'] = df_popular_tags_sample['Title'].apply(lambda txt:clean(txt, tokenize=False, strict=True))

In [152]:
df_popular_tags_sample.head(3)

,Title,Body,Tags,Id,Score,ViewCount,AnswerCount,nb_tags,clean_Tags,no_emoj_Title,contain_pop_tag,clean_Title
19757,How to loop through objects in php,"<p>I am not too familiar with objects, i usual...",<php><arrays><object><loops><foreach>,8664208,17,67234,1,5,"[php, arrays, object, loops, foreach]",How to loop through objects in php,True,object php
25990,Nest.Js Redirect from a controller to another,<p>So I have this module:</p>\n\n<pre><code>@M...,<javascript><node.js><typescript><express><nes...,56397866,17,32440,2,5,"[javascript, node.js, typescript, express, nes...",Nest.Js Redirect from a controller to another,True,nest js redirect controller
36207,ODataController returning HTTP 406 Not Acceptable,<p>I'm building an OData 3 service on Web API ...,<c#><.net><asp.net-web-api><odata><http-status...,29975653,9,8263,1,5,"[c#, .net, asp.net-web-api, odata, http-status...",ODataController returning HTTP 406 Not Acceptable,True,http


In [153]:
df_popular_tags_sample['clean_Body'] = df_popular_tags_sample['Body'].apply(lambda txt:clean(txt, tokenize=False, strict=True))